**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
import numpy as np
from packaging.version import parse as parse_version

if parse_version(np.__version__) > parse_version('1.20.0'):
    !{sys.executable} -m pip install lapjv
else:
    !{sys.executable} -m pip install lapjv==1.3.12

!{sys.executable} -m pip install umap-learn facenet-pytorch
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

# Install google-images-download
!{sys.executable} -m pip install git+https://github.com/Joeclinton1/google-images-download.git

# download the ultralytics bing scraper
# from class_utils.download import download_file_maybe_extract
# download_file_maybe_extract(
#     "https://raw.githubusercontent.com/ultralytics/google-images-download/master/bing_scraper.py",
#     directory="."
# )

# install dependencies for ultralytics bing_scraper
# !{sys.executable} -m pip install tqdm selenium
# !apt update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')
# from selenium import webdriver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from google_images_download.google_images_download import googleimagesdownload
from class_utils import make_montage, plot_bboxes
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from lapjv import lapjv
from umap import UMAP
from PIL import Image
import numpy as np
import glob
import os
import shutil

from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
import torch

In [ ]:
#@title -- Downloading Data -- { display-mode: "form" }
from class_utils.download import download_file_maybe_extract
download_file_maybe_extract("https://www.dropbox.com/s/i5kxprnwuqg4s95/george_martin_example.jpg?dl=1", directory="data")

# also create a directory for storing any outputs
import os
os.makedirs("output", exist_ok=True)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

def download_images(keyword, limit, num_retries=5,
                    output_directory='downloads',
                    image_directory='.'):
    for i in range(num_retries):
        response = googleimagesdownload()
        download = response.download(
            {"keywords": k, "limit": 25,
             "output_directory": output_directory,
             "image_directory": image_directory})
        absolute_image_paths = list(download[0].values())[0]

        if len(absolute_image_paths) > 0:
            break

# def download_images(keyword, limit, num_retries=5,
#                     output_directory='downloads',
#                     image_directory='.', chromedriver=None):
#     if chromedriver is None:
#         chromedriver = '/usr/lib/chromium-browser/chromedriver'

#     command = ('python3 bing_scraper.py --search "{keyword:}" --limit 25 ' + 
#               '--download --chromedriver {chromedriver:} ' +
#               '--output_directory="{output_directory:}" ' +
#               '--image_directory="{image_directory:}" ' 
#     ).format(keyword=keyword, chromedriver=chromedriver,
#              output_directory=output_directory,
#              image_directory=image_directory)

#     !$command

def get_image_filenames(
    directory,
    image_exts = ['.jpg', '.jpeg', '.png', '.gif'],
    recursive = True
):
    images = []
    
    for fname in glob.glob(os.path.join(directory, "**/*"), recursive=recursive):
        if os.path.isfile(fname) and os.path.splitext(fname)[-1] in image_exts:
            images.append(fname)
    
    return images

# def enforce_maxres(img, maxres):
#     width, height = img.width, img.height
    
#     if width > height:
#         if width > maxres:
#             width, height = maxres, maxres / width * height
#             img = img.resize((int(width), int(height)), resample=3)
#     else:
#         if height > maxres:
#             width, height = maxres / height * width, maxres
#             img = img.resize((int(width), int(height)), resample=3)
            
#     return img

def plot_clusters(face_extracted, clusts, labelIDs,
                  verbose=1, figsize=(10, 8),
                  show_title=True):
    figures = []

    # loop over the unique face integers
    for labelID in labelIDs:
        # find all indexes into the `data` array that belong to the
        # current label ID, then randomly sample a maximum of 25 indexes
        # from the set
        if verbose:
            print("Faces for face ID: {}".format(labelID))
        
        idxs = np.where(clusts == labelID)[0]
        idxs = np.random.choice(idxs, size=min(25, len(idxs)),
            replace=False)
    
        # initialize the list of faces to include in the montage
        faces = np.asarray([face_extracted[i] for i in idxs])
    
        # create a montage of the faces
        if len(faces):
            montage = make_montage(faces, 5)
        else:
            montage = np.zeros((64, 64, 3))
        
        # show the output montage
        title = "Face ID #{}".format(labelID)
        title = "Unknown Faces" if labelID == -1 else title
        
        fig = plt.figure(figsize=figsize)
        plt.imshow(montage)
        plt.axis('off')

        if show_title:
            plt.title(title)

        figures.append(fig)

    return figures

def plot_faces(face_extracted, poses, w=0.08, h=0.08, ax=None):
    ax = plt.gca()
    
    for i in range(len(face_extracted)):
        face = face_extracted[i]
        pos = poses[i]
        ax.imshow(face, extent=[pos[0] - w/2, pos[0] + w/2,
                                pos[1] - h/2, pos[1] + h/2])

    plt.xlim([np.min(poses[:, 0]) - w, np.max(poses[:, 0]) + w])
    plt.ylim([np.min(poses[:, 1]) - h, np.max(poses[:, 1]) + h])

## Zhlukovanie ľudských tvárí

V tomto notebook-u ukážeme, ako sa dá v Python-e jednoducho realizovať zhlukovanie ľudských tvárí.

---
### Úloha 1: Stiahnutie obrázkov

Ak chceme vykonať zhlukovanie tvárí, budeme samozrejme potrebovať obrázky tvárí. Použijeme preto balíček `googleimagesdownload`, pomocou ktorého si stiahneme niekoľko obrázkov zo služby Google Images – napríklad fotografie známych osobností.

**Do zoznamu  `keywords` v nasledujúcej bunke pridajte mená 5 alebo 6 známych osobností. Použijú sa ako kľúčové slová pri vyhľadávaní fotografií.** 

---


In [ ]:
keywords = [

    
    
    # XXXXXXXX
    
    
    
]

Pre každé z kľúčových slov teraz stiahneme niekoľko obrázkov a uložíme ich v priečinku `downloads`.



In [ ]:
# make sure that the dataset directory is clean before we start
shutil.rmtree('dataset', ignore_errors=True)

# download 25 images for each keyword
for k in keywords:
    download_images(keyword=k, limit=25, output_directory='dataset')

Ak by sťahovanie obrázkov z nejakého dôvodu zlyhalo, odkomentujte nasledujúcu bunku a stiahnite si už hotovú dátovú množinu.



In [ ]:
# !wget -nc -O faceclust_dataset.zip https://www.dropbox.com/s/3mkdxof3r4rmmf2/faceclust_dataset.zip?dl=1
# !unzip -oq -d dataset faceclust_dataset.zip

### Extrakcia a transformácia tvárí

Keďže obrázky, ktoré v príklade používame, sú stiahnuté priamo z Google Images, budú obsahovať nielen ľudské tváre, ale aj celé osoby a ďalšie objekty. Musíme z nich preto nejakou metódou tváre extrahovať. Ak sme vyššie zvolili možnosť `detection_method = "cnn"`, použije sa na to metóda založená na konvolučných neurónových sieťach. Pozrime sa na jednoduchom príklade, ako by mohli výsledky detekcie vyzerať.



In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

mtcnn = MTCNN(
    min_face_size=64,
    device=device,
    keep_all=True,
    post_process=False
)

Teraz si sieť vyskúšajme na vzorovom obrázku. Najprv si ho načítame a následne na ňom spustíme funkciu `mtcnn.detect`. Čo získame, je zoznam obsahujúci ohraničujúce obdĺžniky jednotlivých tvárí a zoznam zodpovedajúcich konfidenčných skóre (0 pre sieť si vôbec nie je istá predikciou; 1 pre sieť si je úplne istá predikciou).

Napokon si ohraničujúce obdĺžniky vykreslíme na obrázok, aby sme sa presvedčili, že všetko správne funguje.



In [ ]:
img = Image.open("data/george_martin_example.jpg")
bboxes, probs = mtcnn.detect(img)

for i, (bbox, prob) in enumerate(zip(bboxes, probs)):
    print(f"Bounding box {i}: {bbox}; score: {prob}")

fig = plot_bboxes(img, bboxes)
fig.savefig("output/face_detection_cnn.jpg",
            bbox_inches="tight", pad_inches=0)

Náš `mtcnn` objekt sa dá použiť aj na extrakciu samotných obrázkov tvárí z nášho celkového obrázka. Spustime si teda funkciu `mtcnn.extract` a zobrazme si mriežku extrahovaných tvárí.



In [ ]:
extracts = mtcnn.extract(img, list(bboxes), None) / 255.0
face_montage = make_montage(extracts.permute(0, 2, 3, 1), 3)
plt.imshow(face_montage); plt.axis('off');

V ďalšom kroku použijeme inú hlbokú sieť na transformáciu obrázkov tvárí do novej reprezentácie, ktorá lepšie vyjadruje podobnosti a rozdiely medzi ľudskými tvárami, než by to dokázala surová pixelová reprezentácia. Sieť bola predtrénovaná ako klasifikátor na dátovej množine s veľkým množstvom ľudských tvárí (VGGFace2).

Po predtrénovaní siete z nej odstránime poslednú vrstvu. Sieť teda následne transformuje každý vstupný obrázok na 512-rozmerný embedovací vektor. Tieto vektory budeme nižšie používať ako reprezentáciu obrázkov tvárí.



In [ ]:
resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

Ďalej získame zoznam všetkých obrázkov prítomných v priečinku "dataset". Prejdeme postupne všetky z nich a budeme z mich extrahovať tváre, ktoré si uložíme do tenzora `face_extracted`. Kópiu tohto tenzora si uložíme aj ako numpy pole `face_extracted_img`: tú použijeme keď budeme tváre vykresľovať. Na tenzor `face_extracted` potom aplikujeme transformáciu `fixed_image_standardization`, ktorá ho transformuje do podoby, akú očakáva neurónová sieť.

Všimnite si, že vykonanie tejto bunky bude trvať pomerne dlho, pretože niektoré obrázky budú mať pravdepodobne pomerne vysoké rozlíšenie a každý z nich musí prejsť sieťou.



In [ ]:
img_paths = get_image_filenames('dataset')
face_extracted = []

for i, img_path in enumerate(img_paths):
    print(f"Extracting from image {i}/{len(img_paths)}: '{img_path}'.")
    img = Image.open(img_path).convert(mode="RGB")
    with torch.no_grad():
        extracts = mtcnn(img)

    if not extracts is None:
        face_extracted.append(extracts)

face_extracted = torch.vstack(face_extracted)
face_extracted_img = face_extracted.permute(0, 2, 3, 1).cpu().numpy() / 255.0
face_extracted = fixed_image_standardization(face_extracted).to(device)

Keď sme teda extrahovali všetky tváre, budeme už pracovať len s malými obrázkami, ktoré budú navyše mať rovnaké štandardné rozmery, takže z nich bude možné vytvoriť dávky. Nasledujúca bunka, v ktorej na každý z obrázkov aplikujeme embedovaciu sieť, by sa vďaka tomu mala vykonať podstatne rýchlejšie než tá predchádzajúca (najmä ak sa používa GPU).

Všimnite si, že tu používame `torch.no_grad`: keďže neplánujeme robiť spätné šírenie, nepotrebujeme v doprednom behu siete konštruovať výpočtový graf – to nám tiež ušetrí trochu času.



In [ ]:
batch_size = 64
face_embeddings = []

for i in range(0, len(face_extracted), batch_size):
    with torch.no_grad():
        embedding = resnet(face_extracted[i:min(i+batch_size, len(face_extracted))])
    face_embeddings.append(embedding)

face_embeddings = torch.vstack(face_embeddings).cpu().numpy()

Platí, že na extrahovaných obrázkoch sa môžu vyskytovať aj tváre iných osôb, než sme predpokladali (na pôvodných fotografiách mohli byť aj ďalší ľudia). Niektoré tváre môžu byť extrahované chybne, alebo sieť môže omylom namiesto tváre extrahovať inú časť fotografie. Uvidíme, ako sa s tým sieť extrahujúca 128-rozmerné reprezentácie vysporiada.



---
### Úloha 2: Zhlukovanie

**Vykonajte zhlukovanie na poli `encodings`, napr. pomocou  metódy DBSCAN. Výsledné čísla zhlukov priraďte do premennej clusts. Nezabudnite, že aby ste dostali dobré výsledky, môže byť potrebné vhodne nastaviť hyperparameter `eps`.** 

---


In [ ]:


# vykonajte zhlukovanie


clusts =      # sem priraďte čísla zhlukov




### Zobrazenie výsledkov

Nakoniec vizualizujeme tváre patriace do jednotlivých zhlukov. Prvý obrázok predstavuje tváre, ktoré nepatria do žiadneho zhluku.



In [ ]:
labelIDs = np.unique(clusts)
numUniqueFaces = len(np.where(labelIDs > -1)[0])
print("Počet rozličných tvárí: {}".format(numUniqueFaces))
print("Fotografie sme hľadali podľa {} rozličných kľúčových slov.".format(len(keywords)))

In [ ]:
figs = plot_clusters(face_extracted_img, clusts, labelIDs, verbose=0)

for ifig, fig in enumerate(figs):
    fig.savefig("output/clust_{}.pdf".format(ifig),
                bbox_inches="tight", pad_inches=0)

---
### Úloha 3: Znižovanie rozmeru pomocou UMAP

**Použite metódu UMAP, aby ste znížili rozmer dát v poli `encodings` zo 128 na 2 – aby sa dáta dali vizualizovať. Aby bol obrázok dobre čitateľný, môže byť potrebné mierne vyladiť argumenty `min_dist` a `spread` (t.j. aby sa tváre príliš neprekrývali a pod.). Výsledné dáta uložte do poľa s názvom `embeds`.** 

---


In [ ]:



# vykonajte znižovanie rozmeru dát




embeds =          # sem priraďte dáta so zníženým rozmerom





Dáta zníženého rozmeru normalizujeme do rozsahu <0, 1>.



In [ ]:
embeds -= embeds.min(axis=0)
embeds /= embeds.max(axis=0)

Vykreslíme tváre na embedovacích pozíciách.



In [ ]:
plt.figure(figsize=(10, 8))
plot_faces(face_extracted_img, embeds)
plt.xlabel('$d_1$')
plt.ylabel('$d_2$')

plt.savefig("output/faces_umap.pdf",
            bbox_inches="tight", pad_inches=0)

### Zobrazenie v mriežke pomocou algoritmu Jonker-Volgenant

V rámci vizualizácie vytvorenej pomocou UMAP vidno vzdialenosti medzi zhlukmi tvárí a podobne. Obrázky sa však navzájom prekrývajú, čo robí obrázok ťažko čitateľným. Preto pozície skúsime premietnuť do pravidelnej mriežky pomocou algoritmu Jonker-Volgenant.



In [ ]:
sqrt_size = int(np.ceil(np.sqrt(len(embeds))))
size = sqrt_size * sqrt_size
grid = np.dstack(np.meshgrid(np.linspace(0, 1, sqrt_size), np.linspace(0, 1, sqrt_size))).reshape(-1, 2)

padded_embeds = np.zeros((size, embeds.shape[1]))
padded_embeds[:embeds.shape[0], :] = embeds

cost_matrix = cdist(grid, padded_embeds, "sqeuclidean").astype(np.float32)
cost_matrix = cost_matrix * (100000 / cost_matrix.max())
row_as, col_as, _ = lapjv(cost_matrix)
grid_jv = grid[col_as]

Nové pozície fotografií uložené v poli `grid_jv` použijeme na vykreslenie.



In [ ]:
plt.figure(figsize=(12, 12))
plot_faces(face_extracted_img, grid_jv)
plt.axis('off')
plt.savefig("output/faces_grid.pdf",
            bbox_inches="tight", pad_inches=0)